# Neural-ODE Continuous Normalizing Flow demo (two-moons)


In [ ]:
!pip install --quiet torchdiffeq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.2 MB/s eta 0:00:00


In [ ]:
# STEP 2 – Clone the ffjord repo and tell Python where to find it
!git clone --depth 1 https://github.com/rtqichen/ffjord
import sys
sys.path.append('/content/ffjord')   # enables  "import ffjord" later
print("Repo cloned.")


Cloning into 'ffjord'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 85 (delta 8), reused 49 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (85/85), 1.15 MiB | 17.92 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Repo cloned.


In [ ]:
# STEP 3  –  Train a CNF on the two-moons toy set
SCRIPT   = '/content/ffjord/train_toy.py'   # current filename in repo
DATASET  = 'moons'          # moons | rings | 8gaussians | …
NITERS   = 5000             # keep it quick for the poster
VIZFREQ  = 5000             # save a figure only at the very end

print(f"Running CNF on {DATASET} for {NITERS} iterations…\n")
!python {SCRIPT} \
        --data {DATASET} \
        --niters {NITERS} \
        --viz_freq {VIZFREQ} \
        --save ./run_moons


Streaming output truncated to the last 5000 lines.
Iter 0062 | Time 0.2092(0.2204) | Loss 3.269259(3.356474) | NFE Forward 20(19.3) | NFE Backward 20(19.2) | CNF Time 0.5000(0.5000)
Iter 0063 | Time 0.2114(0.2197) | Loss 3.254606(3.349343) | NFE Forward 20(19.3) | NFE Backward 20(19.3) | CNF Time 0.5000(0.5000)
Iter 0064 | Time 0.2069(0.2188) | Loss 3.285140(3.344849) | NFE Forward 20(19.4) | NFE Backward 20(19.3) | CNF Time 0.5000(0.5000)
Iter 0065 | Time 0.2062(0.2180) | Loss 3.282549(3.340488) | NFE Forward 20(19.4) | NFE Backward 20(19.4) | CNF Time 0.5000(0.5000)
Iter 0066 | Time 0.2911(0.2231) | Loss 3.269163(3.335495) | NFE Forward 20(19.5) | NFE Backward 20(19.4) | CNF Time 0.5000(0.5000)
Iter 0067 | Time 0.2786(0.2270) | Loss 3.280347(3.331635) | NFE Forward 20(19.5) | NFE Backward 20(19.5) | CNF Time 0.5000(0.5000)
Iter 0068 | Time 0.2625(0.2294) | Loss 3.280064(3.328025) | NFE Forward 20(19.5) | NFE Backward 20(19.5) | CNF Time 0.5000(0.5000)
Iter 0069 | Time 0.2584(0.2315) 

In [ ]:
# Replace the removed set_axis_bgcolor→set_facecolor in the viz script
!sed -i "s/set_axis_bgcolor/set_facecolor/g" /content/ffjord/diagnostics/viz_toy.py


In [ ]:
import os, torch

# 1) Make sure the ffjord repo is on your PYTHONPATH
import sys
sys.path.append('/content/ffjord')

# 2) Imports
from lib.toy_data import inf_train_gen
from diagnostics.viz_toy import save_trajectory, trajectory_to_video
from train_misc import build_model_tabular, create_regularization_fns

# 3) Load your trained checkpoint
ckpt = torch.load(
    'run_moons/checkpt.pth',
    map_location='cpu',
    weights_only=False  # allow full pickle unpickling
)
args = ckpt['args']

# 4) Rebuild & load the model
model = build_model_tabular(args, 2, create_regularization_fns(args)[0])
model.load_state_dict(ckpt['state_dict'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device).eval()

# 5) Draw a big batch of data from the moons generator
data_samples = inf_train_gen(args.data, batch_size=2000)

# 6) Dump out the trajectory frames
traj_dir = 'run_moons/trajectory'
os.makedirs(traj_dir, exist_ok=True)
save_trajectory(model, data_samples, traj_dir, device=device)

# 7) (Optional) stitch them into a video (MP4/GIF)
trajectory_to_video(traj_dir)

print("✅ Trajectory frames and video are in:", traj_dir)


/content/ffjord/lib/layers/odefunc.py:288: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t).type_as(y)
/content/ffjord/lib/layers/odefunc.py:288: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t).type_as(y)
/content/ffjord/lib/layers/odefunc.py:288: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t).type_as(y)
/content/ffjord/lib/layers/odefunc.py:288: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().require

✅ Trajectory frames and video are in: run_moons/trajectory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


ValueError: mount failed

In [ ]:
!cp -r run_moons /content/drive/MyDrive/NeuralODE_CNF_Demo



